In [1]:
import pyspark
from pyspark.sql import SparkSession

#Create SparkSession
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("spark_juracy") \
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .getOrCreate()
sc = spark.sparkContext
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

/home/juracybertoldo2/.local/lib/python3.8/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob
from pyspark.sql.types import *

/tmp/ipykernel_169473/2205588029.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [31]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.domain",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

In [ ]:
# start 13:15

In [32]:
path = "/dados10t/datalake/raw/omop/dados_auxiliares_de_input/vocabulario/vocabulario_update_16102023/"

desabilitando constrains diretamente via sql no dbaver 

ALTER TABLE omop54.vocabulary drop CONSTRAINT fpk_vocabulary_vocabulary_concept_id


ALTER TABLE omop54.concept drop CONSTRAINT fpk_concept_concept_class_id;

ALTER TABLE omop54.concept drop CONSTRAINT fpk_concept_domain_id;

ALTER TABLE omop54.concept drop CONSTRAINT fpk_concept_vocabulary_id;


ALTER TABLE omop54.concept_ancestor  drop CONSTRAINT fpk_concept_ancestor_ancestor_concept_id;

ALTER TABLE omop54.concept_ancestor drop CONSTRAINT fpk_concept_ancestor_descendant_concept_id;

### tb vocabulary

In [33]:
vocab = spark.read.csv(path+"VOCABULARY.csv",sep="\t",header=True,inferSchema=True)

In [34]:
vocab.printSchema()

root
 |-- vocabulary_id: string (nullable = true)
 |-- vocabulary_name: string (nullable = true)
 |-- vocabulary_reference: string (nullable = true)
 |-- vocabulary_version: string (nullable = true)
 |-- vocabulary_concept_id: integer (nullable = true)



In [35]:
vocab.count()

63

In [10]:
#vocab.select("vocabulary_name").show(100,False)

In [36]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.vocabulary",
    "user": "postgres",
    "password": "omop2023",
}
#    "postgreSQLConf": "constraint.mode=noconstraints"


vocab.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### conceito

In [37]:
concept = spark.read.csv(path+"CONCEPT.csv",sep="\t",header=True,inferSchema=False)

In [38]:
concept.count() #5954357

6206230

In [39]:
concept = concept.withColumn("valid_start_date",F.to_date("valid_start_date",format="yyyyMMdd") )
concept = concept.withColumn("valid_end_date",F.to_date("valid_end_date",format="yyyyMMdd") )
concept = concept.withColumn("concept_id",F.col("concept_id").cast(IntegerType()))

In [40]:
#df.select("valid_start_date","valid_end_date").show()

In [41]:
concept.count()

6206230

In [42]:
concept.printSchema()

root
 |-- concept_id: integer (nullable = true)
 |-- concept_name: string (nullable = true)
 |-- domain_id: string (nullable = true)
 |-- vocabulary_id: string (nullable = true)
 |-- concept_class_id: string (nullable = true)
 |-- standard_concept: string (nullable = true)
 |-- concept_code: string (nullable = true)
 |-- valid_start_date: date (nullable = true)
 |-- valid_end_date: date (nullable = true)
 |-- invalid_reason: string (nullable = true)



In [43]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.concept",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

In [44]:
concept.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### conceito ancestor

In [45]:
conc_ancestor = spark.read.csv(path+"CONCEPT_ANCESTOR.csv",sep="\t",header=True,inferSchema=True)

In [46]:
conc_ancestor.count() #72501431

75834327

In [47]:
conc_ancestor.printSchema()

root
 |-- ancestor_concept_id: integer (nullable = true)
 |-- descendant_concept_id: integer (nullable = true)
 |-- min_levels_of_separation: integer (nullable = true)
 |-- max_levels_of_separation: integer (nullable = true)



In [48]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.concept_ancestor",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


conc_ancestor.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

###  domain

In [49]:
domain = spark.read.csv(path+"DOMAIN.csv",sep="\t",header=True,inferSchema=True)

In [50]:
domain.printSchema()

root
 |-- domain_id: string (nullable = true)
 |-- domain_name: string (nullable = true)
 |-- domain_concept_id: integer (nullable = true)



In [51]:
domain.count()

50

In [52]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.domain",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


domain.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### conceito class

In [53]:
conc_class = spark.read.csv(path+"CONCEPT_CLASS.csv",sep="\t",header=True,inferSchema=True)

In [54]:
conc_class.count()

417

In [55]:
conc_class.printSchema()

root
 |-- concept_class_id: string (nullable = true)
 |-- concept_class_name: string (nullable = true)
 |-- concept_class_concept_id: integer (nullable = true)



In [56]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.concept_class",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


conc_class.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### relationship

In [57]:
relationship = spark.read.csv(path+"RELATIONSHIP.csv",sep="\t",header=True,inferSchema=True)

In [58]:
relationship.count()

690

In [59]:
relationship.printSchema()

root
 |-- relationship_id: string (nullable = true)
 |-- relationship_name: string (nullable = true)
 |-- is_hierarchical: integer (nullable = true)
 |-- defines_ancestry: integer (nullable = true)
 |-- reverse_relationship_id: string (nullable = true)
 |-- relationship_concept_id: integer (nullable = true)



In [60]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.relationship",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


relationship.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### conceito relationship

In [61]:
conc_rl = spark.read.csv(path+"CONCEPT_RELATIONSHIP.csv",sep="\t",header=True,inferSchema=False)

In [62]:
conc_rl.count()

49203082

In [63]:
conc_rl.printSchema()

root
 |-- concept_id_1: string (nullable = true)
 |-- concept_id_2: string (nullable = true)
 |-- relationship_id: string (nullable = true)
 |-- valid_start_date: string (nullable = true)
 |-- valid_end_date: string (nullable = true)
 |-- invalid_reason: string (nullable = true)



In [64]:
conc_rl = conc_rl.withColumn("valid_start_date",F.to_date("valid_start_date",format="yyyyMMdd") )
conc_rl = conc_rl.withColumn("valid_end_date",F.to_date("valid_end_date",format="yyyyMMdd") )
conc_rl = conc_rl.withColumn("concept_id_1",F.col("concept_id_1").cast(IntegerType()))

conc_rl = conc_rl.withColumn("concept_id_1",F.col("concept_id_1").cast(IntegerType()))
conc_rl = conc_rl.withColumn("concept_id_2",F.col("concept_id_2").cast(IntegerType()))

In [65]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.concept_relationship",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


conc_rl.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### CONCEPT_SYNONYM

In [66]:
conc_synonym = spark.read.csv(path+"CONCEPT_SYNONYM.csv",sep="\t",header=True,inferSchema=True)

In [67]:
conc_synonym.count() #2058520

2204058

In [68]:
conc_synonym.printSchema()

root
 |-- concept_id: integer (nullable = true)
 |-- concept_synonym_name: string (nullable = true)
 |-- language_concept_id: integer (nullable = true)



In [70]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.concept_synonym",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


conc_synonym.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### DRUG_STRENGT

In [72]:
drug_strengt = spark.read.csv(path+"DRUG_STRENGTH.csv",sep="\t",header=True,inferSchema=True)

In [73]:
drug_strengt.printSchema()

root
 |-- drug_concept_id: integer (nullable = true)
 |-- ingredient_concept_id: integer (nullable = true)
 |-- amount_value: double (nullable = true)
 |-- amount_unit_concept_id: integer (nullable = true)
 |-- numerator_value: double (nullable = true)
 |-- numerator_unit_concept_id: integer (nullable = true)
 |-- denominator_value: double (nullable = true)
 |-- denominator_unit_concept_id: integer (nullable = true)
 |-- box_size: integer (nullable = true)
 |-- valid_start_date: integer (nullable = true)
 |-- valid_end_date: integer (nullable = true)
 |-- invalid_reason: string (nullable = true)



In [74]:
drug_strengt.count()

2980115

In [75]:
customSchema_drug_strengt = StructType([
        StructField("drug_concept_id", IntegerType(), True),
        StructField("ingredient_concept_id", IntegerType(), True),
        StructField("amount_value", FloatType(), True),
        StructField("amount_unit_concept_id", IntegerType(), True),
        StructField("numerator_value", FloatType(), True),
        StructField("numerator_unit_concept_id", IntegerType(), True),
        StructField("denominator_value", FloatType(), True),
        StructField("denominator_unit_concept_id", IntegerType(), True),
        StructField("box_size", IntegerType(), True),
        StructField("valid_start_date", StringType(), True),
        StructField("valid_end_date", StringType(), True),
        StructField("invalid_reason", StringType(), True)])

In [76]:
drug_strengt = spark.read.schema(customSchema_drug_strengt).csv(path+"DRUG_STRENGTH.csv",sep="\t",header=True,inferSchema=False)

In [77]:
drug_strengt.printSchema()

root
 |-- drug_concept_id: integer (nullable = true)
 |-- ingredient_concept_id: integer (nullable = true)
 |-- amount_value: float (nullable = true)
 |-- amount_unit_concept_id: integer (nullable = true)
 |-- numerator_value: float (nullable = true)
 |-- numerator_unit_concept_id: integer (nullable = true)
 |-- denominator_value: float (nullable = true)
 |-- denominator_unit_concept_id: integer (nullable = true)
 |-- box_size: integer (nullable = true)
 |-- valid_start_date: string (nullable = true)
 |-- valid_end_date: string (nullable = true)
 |-- invalid_reason: string (nullable = true)



In [78]:
drug_strengt = drug_strengt.withColumn("valid_start_date",F.to_date("valid_start_date",format="yyyyMMdd") )
drug_strengt = drug_strengt.withColumn("valid_end_date",F.to_date("valid_end_date",format="yyyyMMdd") )

In [79]:
drug_strengt.show()

+---------------+---------------------+------------+----------------------+---------------+-------------------------+-----------------+---------------------------+--------+----------------+--------------+--------------+
|drug_concept_id|ingredient_concept_id|amount_value|amount_unit_concept_id|numerator_value|numerator_unit_concept_id|denominator_value|denominator_unit_concept_id|box_size|valid_start_date|valid_end_date|invalid_reason|
+---------------+---------------------+------------+----------------------+---------------+-------------------------+-----------------+---------------------------+--------+----------------+--------------+--------------+
|       40729359|              1337720|        null|                  null|            1.2|                     8576|             null|                       8587|    null|      2018-06-04|    2099-12-31|          null|
|       40713261|              1337720|        null|                  null|           60.0|                     8576|   

In [80]:
drug_strengt.count() #2976841

2980115

In [81]:
options = {
    "url": "jdbc:postgresql://localhost:5432/postgres",
    "dbtable": "omop54_v2.drug_strength",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}


drug_strengt.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

### final script